In [1]:
import os
import shutil
import numpy as np
import matplotlib.pyplot as plt

In [4]:
os.getcwd()

'/home/juntingd/research/Attacker_code/ML'

In [48]:
GAN_VAE=False
noGAN=False
noVAE=True
makePlot=False
savePlot=False
seed=128
snrs=[35, 30, 25, 20, 15, 10, 5]
nsdr=11

assert GAN_VAE != (noGAN or noVAE)
if noGAN:
    path_parent = "../checkpoint2/generator/gen_poly_ray_SDR_normAll_nosnrclf_bn_dup_poly_deg5_mem3"
elif noVAE:
    path_parent = "../checkpoint2/ganWOvae/gan_poly_ray_SDR_normAll_nosnrclf_bn_dup_poly_deg5_mem3_1"
elif GAN_VAE:
    path_parent = "../checkpoint2/gan/gan_poly_ray_SDR_normAll_nosnrclf_bn_dup"

acc_snrs=np.zeros((len(snrs),))
if not os.path.exists(path_parent):
    print("ckp not found.")
else:
    for snri in range(len(snrs)):
        snr = snrs[snri]
        acc_sdrs=np.zeros((nsdr,))
        ckp_sdrs={}
        plot_path = path_parent+"/snr"+str(snr)+"/plot/"
        if savePlot and not os.path.exists(plot_path):
            os.makedirs(plot_path+"/ckp")
            os.makedirs(plot_path+"/time_curve")
        for seed_folder in os.listdir(path_parent+"/snr"+str(snr)):
            if seed_folder[:4] != 'seed':
                continue
            if not makePlot and int(seed_folder[4:])!=seed:
                continue
            path_ckp = path_parent+"/snr"+str(snr)+"/"+seed_folder+"/ckp/"
            files = os.listdir(path_ckp)
            for file in files:
                if len(file.split("_"))<2:
                    continue
                sdr = int(file.split("_")[0][3:])
                if sdr>=nsdr:
                    continue
                if sdr not in ckp_sdrs.keys():
                    ckp_sdrs[sdr]=['', '']
                
                if noGAN:
                    val_acc = float(file.split("_")[4].split('.gan')[0][6:])
                else:
                    val_acc = float(file.split("_")[4][6:])
                if val_acc >= acc_sdrs[sdr]:
                    acc_sdrs[sdr] = max(acc_sdrs[sdr], val_acc)
                    ckp_sdrs[sdr][0] = seed_folder
                    ckp_sdrs[sdr][1] = file
        if savePlot:
            for sdr in range(nsdr):
                shutil.copyfile(path_parent+"/snr"+str(snr)+"/"+ckp_sdrs[sdr][0]+"/ckp/"+ckp_sdrs[sdr][1], plot_path+"ckp/"+ckp_sdrs[sdr][1])
                shutil.copyfile(path_parent+"/snr"+str(snr)+"/"+ckp_sdrs[sdr][0]+"/time_curve/time_curve_"+str(sdr)+".npy", plot_path+"/time_curve/time_curve_"+str(sdr)+".npy")

        acc_snrs[snri]=np.mean(acc_sdrs)
        print("snr:{}, average acc:{}%".format(snr, np.round(100*acc_snrs[snri], 2)))

snr:35, average acc:89.66%
snr:30, average acc:94.13%
snr:25, average acc:96.57%
snr:20, average acc:91.55%
snr:15, average acc:93.31%
snr:10, average acc:94.66%
snr:5, average acc:93.9%


In [ ]:
plt.plot(snrs, np.round(100*acc_snrs, 2))
plt.xlabel("SNR (dB)")
plt.ylabel("FPR (%)")
plt.grid()
plt.show()

In [50]:
import os
os.getcwd()

'/home/juntingd/research/Attacker_code/ML'

In [70]:
import torch
from torchinfo import summary
from GAN_SDR_noVAE import sigGenerator
from GAN_SDR import ClassifierSDR, Discriminator

GAN_VAE = True
noGAN = False
noVAE = False

testPlot = True
SDRlabel = 0

device = 'cuda' if torch.cuda.is_available() else 'cpu'
if noGAN:
    path_parent = "../checkpoint2/generator/gen_poly_ray_SDR_normAll_nosnrclf_bn_dup_poly_deg5_mem3"
elif noVAE:
    path_parent = "../checkpoint2/ganWOvae/gan_poly_ray_SDR_normAll_nosnrclf_bn_dup_poly_deg5_mem3_1"
elif GAN_VAE:
    path_parent = "../checkpoint2/gan/gan_poly_ray_SDR_normAll_nosnrclf_bn_dup"

path = path_parent+"/snr35/plot/ckp/"
ckps = os.listdir(path)
for ckp in ckps:
    if len(ckp.split("_"))<2:
        continue
    sdr_load = int(ckp.split("_")[0][3:])
    if sdr_load == SDRlabel:
        break
SNR_load = int(ckp.split("_")[1][3:])
train_acc = float(ckp.split("_")[3][8:])
val_acc = float(ckp.split("_")[4].split(".")[0][6:])
ckp_load=torch.load(path+ckp, map_location=device)
generator = ckp_load["generator"]
generator.device = device
summary(generator, input_size=(32, 2, 800))

discriminator = ckp_load["discriminator"]
summary(discriminator, input_size=(32, 2, 800))

Layer (type:depth-idx)                   Output Shape              Param #
Discriminator                            [32, 1]                   --
├─Sequential: 1-1                        [32, 1]                   --
│    └─Linear: 2-1                       [32, 256]                 409,856
│    └─ReLU: 2-2                         [32, 256]                 --
│    └─Linear: 2-3                       [32, 32]                  8,224
│    └─ReLU: 2-4                         [32, 32]                  --
│    └─Linear: 2-5                       [32, 1]                   33
│    └─Sigmoid: 2-6                      [32, 1]                   --
Total params: 418,113
Trainable params: 418,113
Non-trainable params: 0
Total mult-adds (M): 13.38
Input size (MB): 0.20
Forward/backward pass size (MB): 0.07
Params size (MB): 1.67
Estimated Total Size (MB): 1.95